In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("D:\\datascience\\Copper_project\\copper_data_reg.csv")
df

,Unnamed: 0,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price,selling_price_log,quantity_log,thickness_log
0,0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,2021-04-01,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,2021-07-01,854.00,6.749931,3.991779,0.693147
1,1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,2021-04-01,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,104991,1668701718,2021-04-01,1047.00,6.953684,6.643822,-0.223144
2,2,E140FF1B-2407-4C02-A0DD-780A093B1158,2021-04-01,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,2021-01-01,644.33,6.468211,5.956169,-0.967584
3,3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,2021-04-01,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,2021-01-01,768.00,6.643790,5.310301,0.832909
4,4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,2021-04-01,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,2021-03-01,577.00,6.357842,6.666354,1.386294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181615,181667,0B812591-B501-4F34-859A-B06381FAEB44,2020-07-02,106.783299,30200854.0,25.0,Won,W,41.0,0.96,1240.0,1000222,164141591,2020-07-01,602.00,6.400257,4.670802,-0.040822
181616,181668,DE633116-D1DF-4846-982E-55EFC3658A76,2020-07-02,102.482422,30200854.0,25.0,Won,W,41.0,0.96,1220.0,1000777,164141591,2020-07-01,591.00,6.381816,4.629691,-0.040822
181617,181669,A48374B1-E6DB-45F2-889A-1F9C27C099EB,2020-07-02,208.086469,30200854.0,25.0,Won,W,41.0,0.95,1500.0,1000227,164141591,2020-07-01,589.00,6.378426,5.337954,-0.051293
181618,181670,91643238-5C7B-4237-9A5F-63AE3D35F320,2020-07-02,4.235594,30200854.0,25.0,Won,W,41.0,0.71,1250.0,1004216,164141591,2020-07-01,619.00,6.428105,1.443523,-0.342490


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,181620.0,9.083383e+04,5.244759e+04,0.000000,4.541375e+04,9.081850e+04,1.362642e+05,1.816720e+05
quantity tons,181620.0,5.876596e+03,2.349418e+06,0.000010,1.096871e+01,3.036409e+01,6.716332e+01,1.000000e+09
customer,181620.0,3.023251e+07,1.005651e+05,12458.000000,3.019688e+07,3.020524e+07,3.028042e+07,3.040818e+07
country,181620.0,4.489401e+01,2.440455e+01,25.000000,2.600000e+01,3.000000e+01,7.800000e+01,1.130000e+02
application,181620.0,2.561557e+01,1.775385e+01,2.000000,1.000000e+01,1.500000e+01,4.100000e+01,9.900000e+01
thickness,181620.0,2.550501e+00,2.976518e+00,0.180000,7.000000e-01,1.500000e+00,3.000000e+00,4.000000e+02
width,181620.0,1.295269e+03,2.616509e+02,1.000000,1.179750e+03,1.250000e+03,1.500000e+03,2.990000e+03
product_ref,181620.0,4.740204e+08,7.175358e+08,611728.000000,6.119930e+05,6.406650e+05,1.332077e+09,1.722208e+09
selling_price,181620.0,1.918487e+03,3.318431e+05,0.100000,6.690000e+02,8.120000e+02,9.530000e+02,1.000010e+08
selling_price_log,181620.0,6.626299e+00,5.640581e-01,-2.302585,6.505784e+00,6.699500e+00,6.859615e+00,1.842069e+01


In [4]:
df.dtypes

Unnamed: 0             int64
id                    object
item_date             object
quantity tons        float64
customer             float64
country              float64
status                object
item type             object
application          float64
thickness            float64
width                float64
material_ref          object
product_ref            int64
delivery date         object
selling_price        float64
selling_price_log    float64
quantity_log         float64
thickness_log        float64
dtype: object

In [5]:
def regression(x, y, algorithm):
     
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

    model = algorithm().fit(x_train, y_train)
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)


    accuracy_metrics = {'algorithm': algorithm.__name__,
                        'R2_train' : r2_train,
                        'R2_test'  : r2_test}

    return accuracy_metrics

In [6]:
x = df[['quantity_log','application','thickness_log','width','country','customer','product_ref','status','item type']]
y = df['selling_price_log']
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(x[['item type']])
ohe2 = OneHotEncoder(handle_unknown='ignore')

x_ohe = ohe.fit_transform(x[['item type']]).toarray()
ohe2.fit(x[['status']])
x2 = ohe2.fit_transform(x[['status']]).toarray()

x = np.concatenate((x[['quantity_log', 'application', 'thickness_log', 'width','country','customer','product_ref']].values, x_ohe, x2), axis = 1)

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [7]:
print(regression(x, y, DecisionTreeRegressor))
print(regression(x, y, ExtraTreesRegressor))
print(regression(x, y, RandomForestRegressor))
print(regression(x, y, AdaBoostRegressor))
print(regression(x, y, GradientBoostingRegressor))
print(regression(x, y, XGBRegressor))

{'algorithm': 'DecisionTreeRegressor', 'R2_train': 1.0, 'R2_test': 0.8905794639284016}
{'algorithm': 'ExtraTreesRegressor', 'R2_train': 1.0, 'R2_test': 0.9204283422247893}
{'algorithm': 'RandomForestRegressor', 'R2_train': 0.9895876304627076, 'R2_test': 0.9377918634255464}
{'algorithm': 'AdaBoostRegressor', 'R2_train': 0.6517973464563158, 'R2_test': 0.6426799719876797}
{'algorithm': 'GradientBoostingRegressor', 'R2_train': 0.8905568567078357, 'R2_test': 0.8976168230249889}
{'algorithm': 'XGBRegressor', 'R2_train': 0.9263883619733438, 'R2_test': 0.925610839645335}


In [8]:
print(regression(x, y, RandomForestRegressor))

{'algorithm': 'RandomForestRegressor', 'R2_train': 0.9898596279725408, 'R2_test': 0.9377094282204023}


In [9]:
model = RandomForestRegressor()

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

param_grid = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4],
    'max_features': ['sqrt', 'log2']
}

model = RandomForestRegressor()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)
 
print("Best hyperparameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

metrics_r = {'R2': r2,
           'Mean Absolute Error': mae,
           'Mean Squared Error': mse,
           'Root Mean Squared Error': rmse}

metrics_r


Best hyperparameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5}


{'R2': 0.9314654448768964,
 'Mean Absolute Error': 0.12054882596114332,
 'Mean Squared Error': 0.021989132824410947,
 'Root Mean Squared Error': 0.14828733197549596}

In [10]:
model = RandomForestRegressor(max_depth=20, max_features=None, min_samples_leaf=1, min_samples_split=2).fit(x_train, y_train)
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)


print(r2_train)
print(r2_test)
print(r2_score)

0.9691050650288436
0.9398444374524528
<function r2_score at 0x0000016CE1ADFC40>


In [11]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,181620.0,9.083383e+04,5.244759e+04,0.000000,4.541375e+04,9.081850e+04,1.362642e+05,1.816720e+05
quantity tons,181620.0,5.876596e+03,2.349418e+06,0.000010,1.096871e+01,3.036409e+01,6.716332e+01,1.000000e+09
customer,181620.0,3.023251e+07,1.005651e+05,12458.000000,3.019688e+07,3.020524e+07,3.028042e+07,3.040818e+07
country,181620.0,4.489401e+01,2.440455e+01,25.000000,2.600000e+01,3.000000e+01,7.800000e+01,1.130000e+02
application,181620.0,2.561557e+01,1.775385e+01,2.000000,1.000000e+01,1.500000e+01,4.100000e+01,9.900000e+01
thickness,181620.0,2.550501e+00,2.976518e+00,0.180000,7.000000e-01,1.500000e+00,3.000000e+00,4.000000e+02
width,181620.0,1.295269e+03,2.616509e+02,1.000000,1.179750e+03,1.250000e+03,1.500000e+03,2.990000e+03
product_ref,181620.0,4.740204e+08,7.175358e+08,611728.000000,6.119930e+05,6.406650e+05,1.332077e+09,1.722208e+09
selling_price,181620.0,1.918487e+03,3.318431e+05,0.100000,6.690000e+02,8.120000e+02,9.530000e+02,1.000010e+08
selling_price_log,181620.0,6.626299e+00,5.640581e-01,-2.302585,6.505784e+00,6.699500e+00,6.859615e+00,1.842069e+01


In [12]:
df['item type'].unique()

array(['W', 'WI', 'S', 'Others', 'PL', 'IPL', 'SLAWR'], dtype=object)

In [13]:
test_data = np.array([[1.54, 22, 4.35, 2, 26, 124538, 67548974, 'Won', 'IPL']])

data1 = test_data[:, [0,1,2,3,4,5,6]]
data2 = ohe.transform(test_data[:, [7]]).toarray()
data3 = ohe2.transform(test_data[:, [8]]).toarray()

test_data3 = np.concatenate((data1, data2, data3), axis = 1)

test_data = scaler.transform(test_data3)

pred = best_model.predict(test_data)

print("The predicted selling price is: ", np.exp(pred))

The predicted selling price is:  [516.86916685]


In [14]:
test_data = np.array([[1.54, 22, 4.35, 2, 26, 124538, 67548974, 'Won', 'IPL']])

data1 = ohe.transform(test_data[:, [7]]).toarray()
data2 = ohe2.transform(test_data[:, [8]]).toarray() 
data3 = test_data[:, :6]  

test_data_combined = np.concatenate((test_data[:, [0,1,2,3,4,5,6]], data1, data2), axis=1)

test_data_scaled = scaler.transform(test_data_combined)

pred = best_model.predict(test_data_scaled)

print("The predicted selling price is: ", np.exp(pred))

The predicted selling price is:  [516.86916685]


In [15]:
with open(r"D:\datascience\Copper_project\regression_model.pkl", 'wb') as f:
    pickle.dump(best_model, f)

with open(r"D:\datascience\Copper_project\scaler_reg.pkl", 'wb') as f:
    pickle.dump(scaler, f)

with open(r"D:\datascience\Copper_project\enc_reg.pkl", 'wb') as f:
    pickle.dump(ohe, f)

with open(r"D:\datascience\Copper_project\end_reg2.pkl", 'wb') as f:
    pickle.dump(ohe2, f)

In [16]:
with open(r"D:\datascience\Copper_project\regression_model.pkl", 'rb') as f:
    regr = pickle.load(f)

with open(r"D:\datascience\Copper_project\scaler_reg.pkl", 'rb') as f:
    scaler = pickle.load(f)

with open(r"D:\datascience\Copper_project\enc_reg.pkl", 'rb') as f:
    ohe = pickle.load(f)

with open(r"D:\datascience\Copper_project\end_reg2.pkl", 'rb') as f:
    ohe2 = pickle.load(f)

In [17]:
test_data = np.array([[np.log(40), 10, np.log(250), 0, 28,30202938,1670798778,'PL','Won']])

data1 = ohe.transform(test_data[:, [7]]).toarray()
data2 = ohe2.transform(test_data[:, [8]]).toarray() 
data3 = test_data[:, :6]  

test_data_combined = np.concatenate((test_data[:, [0,1,2,3,4,5,6]], data1, data2), axis=1)

test_data_scaled = scaler.transform(test_data_combined)

y_pred = regr.predict(test_data_scaled)

np.exp(y_pred[0])

SyntaxError: invalid syntax (1618792477.py, line 13)